In [ ]:
from kdrag.all import *
import kdrag.theories.bitvec as bv
import kdrag.theories.seq as seq

SeqBool = seq.Seq(smt.BoolSort())
BitVecMP = kd.Struct("BitVecMP", ("length", smt.IntSort()), ("val", smt.IntSort()))# multiprecision. Like a len + GMP

signed = smt.Function("signed", SeqBool, smt.IntSort())
#smt.Function("unsigned", SeqBool, smt.IntSort())
seqbool = smt.Function("seqbool", smt.IntSort(), smt.IntSort(), SeqBool)
x = smt.Const("x", SeqBool)
kd.axiom(smt.ForAll([x], seqbool(smt.Length(x), signed(x)) == x))

#kd.notation.define("add")

rev = smt.Function("rev", SeqBool, SeqBool)
revrev = kd.axiom(smt.ForAll([x], rev(rev(x)) == x))


#snoc = smt.Function("snoc")

smt.Rev



AttributeError: module 'kdrag.smt' has no attribute 'Rev'

Use smt egraph.

Use BitVec(8) or whatever.
We have a pile of equiational rules we trust (?) work polymorphically. Could also prove them inductively manually in knuckeldragger or lean.
https://gitlab.com/haroldaptroot/haroldbot/-/blob/main/prooffinder.js
Kind of the goal is to extract an equational proof out of the concrete case. An explanation that only uses parametric rules.
We can get clues in so far as what subexpressions might equal others

Maybe this is a general issue. Proof extraction via SMT query. I think I've talk about this before in terms of 
constraint guided prolog
or extracting serquent proof from smt.


Equality proof extraction. Even though we have theories teleporting, maybe we really do want an equational proof extracted.

The codes/semantics distinction is one used all the time in egglog etc. For example extraction as a datalog program. We usually called them terms.



In [ ]:
from kdrag.all import *

# compare to abstract reals
# Kind of an interesting technique. 
# The transfer principle says that everything true over R is true over R*.
# https://en.wikipedia.org/wiki/Transfer_principle
# giving an

def transfer(pf : smt.Proof):
    assert isinstance(thm, kd.Proof)
    return kd.axiom(abstract(thm.thm))

# but in our case we don't want that.
# we want the opposite direction that everything provable in abstract bitvec is provable for specific ones.

def abstract(t : smt.ExprRef): #quote
    T = smt.DeclareSort("abs_" + t.sort().name())
    to = smt.Function("to", t.sort(), T )  # reflect? quote
    fro = smt.Function("fro", T, t.sort()) # interp, std
    # forall x, fro(to(x)) == x
    absdecl = smt.Function(t.decl().name(), *([T]*t.num_args()), T)
    return absdecl(*[abstract(c) for c in t.children()]) # abstract the children
    #return to(fro(t.decl()(*[abstract(c) for c in t.children()])))
    

def extract_eq(t1, t2):
    kd.prove(t1 == t2)
    E1 = EGraph()
    E2 = EGraph()
    #abstract(t1) == t
    # to/from pairs everywhere? it's a galois connection
    E1.add(t1)
    E2.add(t2)
    E.rebuild()
x,y,z = smt.BitVecs("x y z", 8)
abstract(x + y + z)


bvadd(bvadd(x, y), z)

In [ ]:
sorts = {}
def quote_sort(s : smt.SortRef):
    T = smt.DeclareSort("Code<" + s.sexpr() + ">")
    sorts[T] = s
    return T
def interp_ax(s : smt.SortRef):
    T = quote_sort(s)
    interp = smt.Function("interp", T, s) 
    reify = smt.Function("reify", s, T)
    x = smt.Const("x", s)
    return kd.axiom(smt.ForAll([x], interp(reify(x)) == x))
def quote(t : smt.ExprRef):
    assert not isinstance(t, smt.QuantifierRef)
    T = quote_sort(t.sort())
    decl, children = t.decl(), t.children()
    Targs = [quote_sort(c.sort()) for c in children]
    absdecl = smt.Function(decl.name(), *Targs, T)
    return absdecl(*[quote(c) for c in children])
x + y * z
quote(x+y*z).sort()

#def interp(t : smt.ExprRef):

is_true = smt.Function("is_sat", quote_sort(smt.BoolSort()), smt.BoolSort())

def is_true(t : smt.ExprRef, by=[]):
    # is_true is negation is unsat.
    #s = smt.Solver()
    #s.add(smt.Not(interp(t)))
    #if s.check() == smt.sat:
    #    return kd.axiom(is_true(t) == False, reasons=[])
    p = kd.prove(interp(t), by=by)
    return kd.axiom(is_true(t) == True, reasons=[p])
    










    

Code<(_ BitVec 8)>

https://theory.stanford.edu/~barrett/pubs/NPR+21c.pdf Towards Satisfiability Modulo Parametric Bit-vectors
https://youtu.be/WtsInfbzxjs?si=6yXUK1kb_xy4XcEQ 2025 EuroLLVM - How to trust your peephole rewrites: automatically verifying for arbitrary width!
https://michaelblondin.com/automata/files/book_authors.pdf chapter 9 presburger arithmetic

https://dl.acm.org/doi/pdf/10.1145/3485539 FPL: Fast Presburger Arithmetic through Transprecision https://grosser.science/FPL/ 
https://johnwickerson.github.io/papers/luigi_LATTE25.pdf

Try some examples modelling arbitrary bitwidth using streaming algorithms.

Concat is a problem. Liveness somehow gets involved.


I think it does make sense to make zequence 0

Bitvec1 vs Bool. BitVec1 has slightly less impedance mismatch I think.

Maybe I can actually do this by 
SeqFoldI basically.

In [ ]:
from kdrag.all import *
import kdrag.theories.bitvec as bitvec
import kdrag.theories.seq as seq

BV = seq.Seq(smt.BoolSort())

x,y,z = smt.Consts("x y z", BV)

#kd.prove(smt.ForAll([x], smt.Concat(x[0], smt.SubSeq(x, 1, smt.Length(x)-1)) == x))

to_int = smt.Function("to_int", BV, smt.IntSort())
to_int = kd.notation.to_int.define([x], 
smt.If(smt.Length(x) == 0, 
       smt.IntVal(0), 
    x[0] + 
    2 * to_int(smt.SubSeq(x, 1, smt.Length(x)-1))
))


is_empty = kd.define("is_empty", [x], smt.Length(x) == 0)

to_bv = kd.notation.SortDispatch(name="to_bv")
#to_bv.register(smt.BitVecSort(8), )
BV1 = smt.BitVecSort(1)

z = smt.BitVecVal(0,1)

def ToBitVecN(x : smt.BitVecRef) -> smt.SeqRef:
    return smt.Concat(x[i]  for i in range(x.size()))


ToBitVecN(z)

#kd.notation.add.define([x,y], 
#    smt.If(smt.Length(x) == 0, y)
#)


Mixed binary arithmetic


to from asbtract functions

There is an isomorphism here. A useful one.
To ints it's not an isomorphim. We lose the ends.
to the tuple of int and int for length not iso because we there are bad elements.
Under assupmtion of goodness


CF says just add bitwise ops. Nonsense/undefined on negative numbers?

The totalized version is unspecified behavior, not undefined. Apply the same thing twice will get same results. Uh not it has to anyway... Hmm.


| BVOnes int    inverted
| BVZeros int   normal

but then what does arithemtic do on the inverted one?

what about bitwise negation?


oh twos complement. duh

invert add 1

8 = 1000 ->   111110111 + 1 = 11111111000

7 = 0111 -> 11111000 + 1 = 11111001


In [11]:
-8 ^ -7

1

In [ ]:
bvxor = smt.Function("xor", smt.IntSort(), smt.IntSort(), smt.IntSort())
bvand = smt.Function("band", smt.IntSort(), smt.IntSort(), smt.IntSort())
bvor = smt.Function("bor", smt.IntSort(), smt.IntSort(), smt.IntSort())

bvand_comm = kd.axiom(smt.ForAll([x, y], bvand(x, y) == bvand(y, x)))
bvand_assoc = kd.axiom(smt.ForAll([x, y, z], bvand(x, bvand(y, z)) == bvand(bvand(x, y), z)))

# copy over all the rules from the bitvec theory

def schema(n):
    pass
    # ToBitVec(bvxor(x,y)) == ToBitVec(x) ^ ToBitVec(y)
    # and so on.

bvxorpos
bvxor = kd.define("bvxor", [x,y],
    kd.cond(
        (smt.And(x < 0, y < 0), bvxorpos(-x, -y)),
        (smt.And(x < 0, y >= 0), -bvxorpos(-x, y)),
        (smt.And(x >= 0, y < 0), -bvxorpos(x, -y)),
        (smt.And(x >= 0, y >= 0), bvxorpos(x, y)
    )
)